In [125]:
!pip install  pygeohash

In [126]:
import pandas as pd
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
import pygeohash as pgh
from datetime import datetime
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix,f1_score,precision_score,recall_score,precision_recall_curve,accuracy_score,classification_report,plot_precision_recall_curve
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [127]:
df=pd.read_csv("/content/drive/MyDrive/datasets/dataset_TSMC2014_TKY.csv")

In [129]:
df.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,1541,4f0fd5a8e4b03856eeb6c8cb,4bf58dd8d48988d10c951735,Cosmetics Shop,35.705101,139.619590,540,Tue Apr 03 18:17:18 +0000 2012
1,868,4b7b884ff964a5207d662fe3,4bf58dd8d48988d1d1941735,Ramen / Noodle House,35.715581,139.800317,540,Tue Apr 03 18:22:04 +0000 2012
2,114,4c16fdda96040f477cc473a5,4d954b0ea243a5684a65b473,Convenience Store,35.714542,139.480065,540,Tue Apr 03 19:12:07 +0000 2012
3,868,4c178638c2dfc928651ea869,4bf58dd8d48988d118951735,Food & Drink Shop,35.725592,139.776633,540,Tue Apr 03 19:12:13 +0000 2012
4,1458,4f568309e4b071452e447afe,4f2a210c4b9023bd5841ed28,Housing Development,35.656083,139.734046,540,Tue Apr 03 19:18:23 +0000 2012


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573703 entries, 0 to 573702
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   userId           573703 non-null  int64  
 1   venueId          573703 non-null  object 
 2   venueCategoryId  573703 non-null  object 
 3   venueCategory    573703 non-null  object 
 4   latitude         573703 non-null  float64
 5   longitude        573703 non-null  float64
 6   timezoneOffset   573703 non-null  int64  
 7   utcTimestamp     573703 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 35.0+ MB


In [131]:
data = df[["venueCategoryId","latitude","longitude"]]

In [132]:
#num_val= dict(enumerate(data.venueCategoryId.unique()))
#num_val

In [133]:
data['Lat,Long'] = data[['latitude', 'longitude']].apply(tuple, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [134]:
data=data.drop(["latitude","longitude"],axis=1,inplace=False)

In [135]:
df['geohash'] = ""

In [136]:
target=df["venueCategoryId"]

In [137]:

for index,row in df.iterrows():
            value = pgh.encode(row['latitude'],row['longitude'], precision=5)
            df.at[index,"geohash"]=value

In [138]:
var_mod = df[["venueCategory","venueId","geohash","venueCategoryId"]]

In [139]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [140]:
for i in var_mod:
    df[i] = LE.fit_transform(df[i])

In [141]:
print('There are {} uniques categories.'.format(len(df['geohash'].unique())))
df.groupby('geohash')['geohash'].count().sort_values(ascending=False)

There are 81 uniques categories.


geohash
62    88872
40    52392
50    43090
32    35199
31    34937
      ...  
75        9
36        9
76        4
80        2
37        1
Name: geohash, Length: 81, dtype: int64

In [142]:
print(df['venueCategory'].nunique())

247


In [143]:
target

0         4bf58dd8d48988d10c951735
1         4bf58dd8d48988d1d1941735
2         4d954b0ea243a5684a65b473
3         4bf58dd8d48988d118951735
4         4f2a210c4b9023bd5841ed28
                    ...           
573698    4bf58dd8d48988d1e9931735
573699    4bf58dd8d48988d129951735
573700    4bf58dd8d48988d1dc931735
573701    4bf58dd8d48988d16e941735
573702    4bf58dd8d48988d10d951735
Name: venueCategoryId, Length: 573703, dtype: object

In [144]:
df.drop(labels=['utcTimestamp', 'latitude','longitude','userId'],axis=1,inplace=True)

In [145]:
#num_val= dict(map(reversed, num_val.items()))
#num_val

In [146]:
from sklearn.model_selection import train_test_split

In [147]:
X=data.values

In [148]:
y=target

In [149]:
df.columns

Index(['venueId', 'venueCategoryId', 'venueCategory', 'timezoneOffset',
       'geohash'],
      dtype='object')

In [150]:
dataset=df[[ 'venueCategory','venueId', 'venueCategoryId', 'timezoneOffset','geohash']]

In [151]:
dataset=dataset.values

In [152]:
X = dataset[:,1:5].astype(float)
Y = dataset[:,0]

In [153]:
Y

array([ 64, 185,  62, ..., 228,  86, 186])

In [154]:
from tensorflow.keras import utils as np_utils

In [155]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [156]:
#data['venueCategoryId'] = data["venueCategoryId"].apply(lambda x: num_val[x])

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X,dummy_y, test_size=0.33,random_state=2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(384381, 4) (189322, 4) (384381, 247) (189322, 247)


In [158]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model

In [159]:
ndim = X.shape[1]

In [160]:
top_words=1000
embedding_vecor_length=300

In [161]:
dataset.shape

(573703, 5)

In [162]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass


In [163]:
from keras.layers import Dense,Dropout, Activation, Flatten
from keras.regularizers import L2
from keras.layers.core import Dropout

In [164]:
embed_dim = 300
lstm_out = 100
batch_size = 128

model = Sequential()
model.add(Embedding(200, embed_dim,input_length = X.shape[1],embeddings_regularizer="l2"))
model.add(LSTM(lstm_out,activation="relu"))
model.add(Dense(247,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 4, 300)            60000     
                                                                 
 lstm_21 (LSTM)              (None, 100)               160400    
                                                                 
 dense_19 (Dense)            (None, 247)               24947     
                                                                 
Total params: 245,347
Trainable params: 245,347
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history=model.fit(X_train, y_train,validation_data=(X_test,y_test),batch_size = 128, epochs=1, verbose = 2)

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 128)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

In [ ]:
# summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()